<a href="https://colab.research.google.com/github/aryamtos/convnet-tensorflow/blob/master/Processing_Image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:

import cv2
import numpy as np
from PIL import Image
from numpy import asarray
from matplotlib import pyplot as plt
import glob
import pydicom as dicom
from os import rename
from pydicom.encaps import encapsulate
from PIL import Image, ImageFilter, ImageOps
import os, sys
import shutil
import unicodedata
import argparse
import pandas as pd


In [4]:
mlo_muscle = glob.glob("/content/drive/MyDrive/Imagem/*.png")

In [5]:

def select_largest_obj(img_bin, lab_val=255, fill_holes=False, 
                           smooth_boundary=False, kernel_size=15):

        '''Selecione o maior objeto de uma imagem binária e opcionalmente
        preenche os buracos dentro dele e suaviza seu limite.
        Args:
            img_bin (2D array): 2D numpy array da imagem binária.
            lab_val ([int]): integer value -> label (largest 
                    object). Default = 255.
            fill_holes ([boolean]): se preenche os buracos dentro do maior 
                    objeto ou não. Default = false.
            smooth_boundary ([boolean]): se suaviza o limite do 
                    maior objeto usando abertura morfológica ou não.
                    Default = false.
            kernel_size ([int]): o tamanho do kernel usado para operação morfológica.
             Default = 15.
        Returns:
        	Binary mask (:

        '''
        n_labels, img_labeled, lab_stats, _ = \
            cv2.connectedComponentsWithStats(img_bin, connectivity=8, 
                                             ltype=cv2.CV_32S)
        largest_obj_lab = np.argmax(lab_stats[1:, 4]) + 1
        largest_mask = np.zeros(img_bin.shape, dtype=np.uint8)
        largest_mask[img_labeled == largest_obj_lab] = lab_val

        if fill_holes:
            bkg_locs = np.where(img_labeled == 0)
            bkg_seed = (bkg_locs[0][0], bkg_locs[1][0])
            img_floodfill = largest_mask.copy()
            h_, w_ = largest_mask.shape
            mask_ = np.zeros((h_ + 2, w_ + 2), dtype=np.uint8)
            cv2.floodFill(img_floodfill, mask_, seedPoint=bkg_seed, 
                          newVal=lab_val)
            holes_mask = cv2.bitwise_not(img_floodfill)  # mask of the holes.
            largest_mask = largest_mask + holes_mask
        if smooth_boundary:
            kernel_ = np.ones((kernel_size, kernel_size), dtype=np.uint8)
            largest_mask = cv2.morphologyEx(largest_mask, cv2.MORPH_OPEN, 
                                            kernel_)
            
        return largest_mask


In [12]:
for i, index in enumerate(mlo_muscle):
    mammo_org = cv2.imread(index,cv2.IMREAD_GRAYSCALE)
    mammo_med_blurred = cv2.medianBlur(mammo_org,3)
    #print(mammo_med_blurred)
    global_threshold = 18
    _,mammo_binary = cv2.threshold(mammo_org,global_threshold,maxval = 255,type = cv2.THRESH_BINARY +  cv2.THRESH_OTSU)
    mammo_breast_mask = select_largest_obj(mammo_binary,lab_val = 255,
                          fill_holes=False,
                          smooth_boundary=False, kernel_size=5)
    #print(mammo_breast_mask)
    mammo_arti_supr = cv2.bitwise_and(mammo_med_blurred,mammo_breast_mask)
    cv2.imwrite(index,mammo_arti_supr)

In [2]:
!pip install pydicom

     |████████████████████████████████| 1.9MB 5.2MB/s 
